In [1]:
import pandas as pd
import time

In [2]:
with open('onegin.txt') as f:
  phrases = [s.replace('\t\t','') for s in f.read().split('\n') if s.startswith('\t\t')]

In [3]:
phrases

['He мысля гордый свет забавить,',
 'Вниманье дружбы возлюбя,',
 'Хотел бы я тебе представить',
 'Залог достойнее тебя,',
 'Достойнее души прекрасной,',
 'Святой исполненной мечты,',
 'Поэзии живой и ясной,',
 'Высоких дум и простоты;',
 'Но так и быть – рукой пристрастной',
 'Прими собранье пёстрых глав,',
 'Полусмешных, полупечальных,',
 'Простонародных, идеальных,',
 'Небрежный плод моих забав,',
 'Бессонниц, лёгких вдохновений,',
 'Незрелых и увядших лет,',
 'Ума холодных наблюдений',
 'И сердца горестных замет.',
 '«Мой дядя самых честных правил,',
 'Когда не в шутку занемог,',
 'Он уважать себя заставил',
 'И лучше выдумать не мог.',
 'Его пример другим наука;',
 'Но, боже мой, какая скука',
 'С больным сидеть и день и ночь,',
 'Не отходя ни шагу прочь!',
 'Какое низкое коварство',
 'Полуживого забавлять,',
 'Ему подушки поправлять,',
 'Печально подносить лекарство,',
 'Вздыхать и думать про себя:',
 'Когда же чёрт возьмёт тебя!»',
 'Так думал молодой повеса,',
 'Летя в пыли на п

In [4]:
phrases[:10]

['He мысля гордый свет забавить,',
 'Вниманье дружбы возлюбя,',
 'Хотел бы я тебе представить',
 'Залог достойнее тебя,',
 'Достойнее души прекрасной,',
 'Святой исполненной мечты,',
 'Поэзии живой и ясной,',
 'Высоких дум и простоты;',
 'Но так и быть – рукой пристрастной',
 'Прими собранье пёстрых глав,']

In [5]:
text = [[c for c in ph] for ph in phrases if type(ph) is str]

In [6]:
CHARS = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя ')

In [7]:
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]

In [8]:
INDEX_TO_CHAR

['none',
 'е',
 'ю',
 'щ',
 'д',
 'л',
 'р',
 'б',
 'ч',
 'ё',
 'а',
 'ф',
 'п',
 'с',
 'я',
 'г',
 'з',
 'у',
 'й',
 'м',
 'ц',
 'и',
 'о',
 'в',
 'н',
 'ы',
 'т',
 'ъ',
 'к',
 'ш',
 'х',
 'ь',
 'ж',
 'э',
 ' ']

In [9]:
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [10]:
CHAR_TO_INDEX

{'none': 0,
 'е': 1,
 'ю': 2,
 'щ': 3,
 'д': 4,
 'л': 5,
 'р': 6,
 'б': 7,
 'ч': 8,
 'ё': 9,
 'а': 10,
 'ф': 11,
 'п': 12,
 'с': 13,
 'я': 14,
 'г': 15,
 'з': 16,
 'у': 17,
 'й': 18,
 'м': 19,
 'ц': 20,
 'и': 21,
 'о': 22,
 'в': 23,
 'н': 24,
 'ы': 25,
 'т': 26,
 'ъ': 27,
 'к': 28,
 'ш': 29,
 'х': 30,
 'ь': 31,
 'ж': 32,
 'э': 33,
 ' ': 34}

In [11]:
import torch

In [12]:
MAX_LEN = 30

In [13]:
X = torch.zeros((len(text), MAX_LEN), dtype=int)

In [14]:
for i in range(len(text)):
    for j, w in enumerate(text[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [15]:
X[0:1]

tensor([[ 0,  0, 34, 19, 25, 13,  5, 14, 34, 15, 22,  6,  4, 25, 18, 34, 13, 23,
          1, 26, 34, 16, 10,  7, 10, 23, 21, 26, 31,  0]])

In [16]:
class Network(torch.nn.Module):

    def __init__(self):
        super(Network, self).__init__()
        self.word_embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 35)
        self.gru = torch.nn.RNN(35, 128, batch_first=True)
        self.hidden2tag = torch.nn.Linear(128, len(INDEX_TO_CHAR))

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [17]:
class NetworkGRU(torch.nn.Module):

    def __init__(self):
        super(NetworkGRU, self).__init__()
        self.word_embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 35)
        self.gru = torch.nn.GRU(35, 128, batch_first=True)
        self.hidden2tag = torch.nn.Linear(128, len(INDEX_TO_CHAR))

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [18]:
class NetworkLSTM(torch.nn.Module):

    def __init__(self):
        super(NetworkLSTM, self).__init__()
        self.word_embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 35)
        self.lstm = torch.nn.LSTM(35, 128, batch_first=True)
        self.hidden2tag = torch.nn.Linear(128, len(INDEX_TO_CHAR))

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.lstm(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.lstm(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [19]:
class NetworkUber(torch.nn.Module):

    def __init__(self):
        super(NetworkGRU, self).__init__()
        self.word_embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 35)
        self.gru = torch.nn.GRU(35, 128, batch_first=True, num_layers=3)
        self.hidden2tag = torch.nn.Linear(128, len(INDEX_TO_CHAR))

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [20]:
model = Network()

In [21]:
model.forward(X[0:1])[0].shape

torch.Size([1, 30, 35])

In [22]:
def generate_sentence(mdl):
    sentence = ['я', ' ', 'в', 'а', 'с']
    #sentence = ['а', 'х', ' ', 'н', 'о', 'ж', 'к', 'и']
    state = None
    for i in range(MAX_LEN):
        X = torch.Tensor([[CHAR_TO_INDEX[sentence[i]]]]).type(torch.long)
        if i == 0:
            result, state = mdl.forward(X)
        else:
            result, state = mdl.forward_state(X, state)
        prediction = result[0, -1, :]
        index_of_prediction = prediction.argmax()
        if i >= len(sentence) - 1:
            if index_of_prediction == 0:
                break
        sentence.append(INDEX_TO_CHAR[index_of_prediction])
    print(''.join(sentence))

In [23]:
generate_sentence(model)

я василхзаияаиёииёбииии ииии виии и


In [32]:
def learning_loop(mdl, epochs=2000, optimizer=None):
    criterion = torch.nn.CrossEntropyLoss()
    for ep in range(epochs):
        start = time.time()
        train_loss = 0.
        train_passed = 0

        for i in range(int(len(X) / 100)):
            batch = X[i * 100:(i + 1) * 100]
            X_batch = batch[:, :-1]
            Y_batch = batch[:, 1:].flatten()

            optimizer.zero_grad()
            answers, _ = mdl.forward(X_batch)
            answers = answers.view(-1, len(INDEX_TO_CHAR))
            loss = criterion(answers, Y_batch)
            train_loss += loss.item()

            loss.backward()
            optimizer.step()
            train_passed += 1

        print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
        generate_sentence(mdl)

In [ ]:
learning_loop(model, epochs=2000, optimizer=torch.optim.SGD(model.parameters(), lr=.5))

Epoch 0. Time: 0.673, Train loss: 3.046
я вас   nonenone   nonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenone
Epoch 1. Time: 0.658, Train loss: 2.740
я вас       ооооо        ооооо     
Epoch 2. Time: 0.537, Train loss: 2.655
я вас о   о ооо о     оооооо      о
Epoch 3. Time: 0.531, Train loss: 2.606
я вас о   о оооооо     оооооо     о
Epoch 4. Time: 0.528, Train loss: 2.565
я вас о   оооооо     ноооооо     но
Epoch 5. Time: 0.587, Train loss: 2.527
я вас со  оо осонсно     оосссооо  
Epoch 6. Time: 0.620, Train loss: 2.490
я вас со  сооссоо     нссспооооо   
Epoch 7. Time: 0.643, Train loss: 2.455
я вас по осонсно о о ннснсооео     
Epoch 8. Time: 0.532, Train loss: 2.422
я вас полтсонао     сспппооооо     
Epoch 9. Time: 0.585, Train loss: 2.391
я вас полтсонао  е  спнспооооо     
Epoch 10. Time: 0.596, Train loss: 2.362
я вас полтпонао  е  спнспоооео     
Epoch 11. Time: 0.554, Train loss: 2.336
я вас полтпорао  а  спсппооооо     
Epoch 12. Time

In [ ]:
model = NetworkGRU()
learning_loop(model, epochs=2000, optimizer=torch.optim.Adam(model.parameters(), lr=.5))

In [ ]:
model = NetworkLSTM()
learning_loop(model, epochs=2000, optimizer=torch.optim.SGD(model.parameters(), lr=.5))

In [ ]:
model = NetworkUber()
learning_loop(model, epochs=2000, optimizer=torch.optim.SGD(model.parameters(), lr=.5))